In [34]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [35]:
df = pd.read_parquet('data2.parquet')
df.head()

,Data,Replied To
0,Sender: MaryJoy Galvez <donotreply@appfolio.co...,0
1,Sender: Yieldstreet <investments@yieldstreet.c...,0
2,Sender: Amy Lewis <amylewis@parkerremick.com>\...,1
3,Sender: Yieldstreet <investments@yieldstreet.c...,0
4,"Sender: ""FLEE the Ultimate VR Arena & Escape G...",0


In [36]:
print(df.value_counts('Replied To'))

Replied To
0    21232
1     4268
Name: count, dtype: int64


In [37]:
#equalize the class imbalance
df = df.groupby('Replied To').head(4268)
df.value_counts('Replied To')

Replied To
0    4268
1    4268
Name: count, dtype: int64

In [46]:
x = df['Data']
y = df['Replied To']

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200, output_mode='int')
vectorizer.adapt(x)

x = vectorizer(x)
x = keras.preprocessing.sequence.pad_sequences(x, padding='post')

dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=1024).batch(64).prefetch(tf.data.AUTOTUNE).cache()

#print some of the data
for x, y in dataset.take(1):
	print(x)
	print(y)

tf.Tensor(
[[   4   65  182 ...    0    0    0]
 [   4   38   14 ...    0    0    0]
 [   4 8677    3 ...    0    0    0]
 ...
 [   4  411  465 ...    0    0    0]
 [   4 1488 1468 ...    0    0    0]
 [   4 6748 9345 ...    0    0    0]], shape=(64, 200), dtype=int32)
tf.Tensor(
[0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0], shape=(64,), dtype=int64)


In [39]:
# Create the model
model = keras.Sequential([
    layers.Embedding(20000, 128, mask_zero=True),
    layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_8 (LSTM)               (None, 128)               131584    
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [41]:
#create an 80/20 split for training and validation, taking into account that the total data size is 8536    
train_size = int(8536 * 0.8)
val_size = int(8536 * 0.2)

train = dataset.take(train_size)
val = dataset.skip(train_size)

model.fit(
    train,
    validation_data=val,
    epochs=10,
    callbacks=[keras.callbacks.ReduceLROnPlateau(monitor='accuracy', patience=3, verbose=1, factor=.5)]
)

Epoch 1/10
134/134 [==============================] - 71s 529ms/step - loss: 0.0825 - accuracy: 0.9740 - lr: 0.0010
Epoch 2/10
134/134 [==============================] - 71s 527ms/step - loss: 0.1296 - accuracy: 0.9565 - lr: 0.0010
Epoch 3/10
134/134 [==============================] - 72s 536ms/step - loss: 0.1122 - accuracy: 0.9639 - lr: 0.0010
Epoch 4/10
134/134 [==============================] - ETA: 0s - loss: 0.1023 - accuracy: 0.9679
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
134/134 [==============================] - 72s 535ms/step - loss: 0.1023 - accuracy: 0.9679 - lr: 0.0010
Epoch 5/10
134/134 [==============================] - 73s 542ms/step - loss: 0.0503 - accuracy: 0.9888 - lr: 5.0000e-04
Epoch 6/10
134/134 [==============================] - 73s 545ms/step - loss: 0.0462 - accuracy: 0.9879 - lr: 5.0000e-04
Epoch 7/10
134/134 [==============================] - 74s 553ms/step - loss: 0.0362 - accuracy: 0.9910 - lr: 5.0000e-04
Epoch 8/10
134/